## MSING0097 Group Coursework (Notebook 2) - F5:  

## Time series data continued

In this Notebook you must conduct a similar analysis to the one you worked through in Notebook 1.

To do this we have selected several sources of data, from recent reserach papers and commercial challenge prizes, that you can from the basis of your analysis. 

## MSIN0097: Predictive Analytics (18/19), Group Project, Team Members, Team F5:
- Daria Golova (17145084)<br>
- Bartosz Kultys (18062858)<br>
- Chi Chung Yuen (Kelvin) (15027396)<br>
- Nanistya Respati Probosutedjo (1713278)<br>
- Tjark Petersen (17130626)<br>

<div class="alert alert-warning">
<b>COURSE ASSESSMENT (~1200 words)</b> 

<p>This part of the assessment is based on answering/commenting/making observations on various sections throughout this Notebook.<p> 

<p>Course assessment blocks are highlighted in  orange (same color as this cell) throughout the Notebook.<p> 
<p>Please follow the instructions in each block. Please look at this markup cell and format your responses in a similar manner</p>

<b> NOTE: All comments will be made in blue markup cells for uniformity and ease of identification in unexpected areas, accompanied with word count. 

</div>


## Table of Contents - Project Template

 Load libraries

1. [Problem Comprehention](#problem)
    * Overview
    * Problem Description
 
 
2. [Data Comprehention](#data)
    * Descriptive statistics
    * Data visualizations
  
  
3. [Data Preparation](#prep)
    * Feature engineering
    * Data transformation (normalisation, balancing, spliting)
  
  
4. [Fitting the Algorithms](#alg)
    * Experimental setup
    * Naive forecast
    * Data analysis
    * Linear models 
    * Supervised learning formulation
    * Ensemble methods
    * Advanced methods
  
  
5. [Evaluation](#assesment)
  
  
6. [Finalize Model and Summary](#final)


In [ ]:
!pip install keras
!pip install arch

In [ ]:
# Import all required libraries
import pandas as pd
from pandas.tools.plotting import lag_plot

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import random
from math import sqrt

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error

#from arch import arch_model #for GARCH

from scipy.stats import probplot, shapiro

from statsmodels.stats.diagnostic import het_arch
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.diagnostic import het_arch
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Conv2D, MaxPooling2D
from keras import backend as K

#View all columns of dataframes
pd.options.display.max_columns = None

#View all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

#pretty printing of dataframes in Jupyter notebook
import IPython
from IPython import display


<br>
<div class="alert alert-success">
<h1 align="center"> 1. Problem Comprehension </h1> <br>
<a id="problem"></a>

## 0. Overview

Please set out the selected data set/s you will use for your analysis.

In [ ]:
%%time
# load the data
#d = pd.read_csv('household_power_consumption.txt', sep=';')

In [ ]:
%%time
# The first value of Sub_metering_1 is 0 whoich will lead to an ffill of 0 later on, quick solution:
d.Sub_metering_1 = d.Sub_metering_1.convert_objects(convert_numeric=True)
d.iloc[0,6] = d.Sub_metering_1.mean()
# as seen above, missing values are marked with "?", % of nan values, then median imputation
df = d
df = df.replace(to_replace='?', value= np.nan)
df = df.replace(to_replace= '0.000', value= np.nan)
df = df.replace(to_replace= '0', value= np.nan)
df = df.replace(to_replace= '0.0', value= np.nan)
df = df.replace(to_replace= 0, value= np.nan)

def check_null(data):
    nlist = list()
    for col in data.columns:
        nlist.append(len([i for i in data[col].isnull() if i==True])/data.shape[0])
    labels = [i for i in data.columns]
    d_null=pd.DataFrame({'Columns': labels, 'Null%': nlist})
    return d_null

check_null(df)

# NAN values but only 1.25% in each column,impute forward
df.fillna(method = 'ffill', inplace=True)
print('Imputation done.')

# check whether imputation is complete
print(check_null(df))

# make numbers floats
d = df.convert_objects(convert_numeric=True)

# check whether dtype has been converted to float
print(d.info())

In [ ]:
%%time
# deal with date time format - takes couple of minutes though but gives us a time series element (datetime object) which is useful:
d.loc[:,'log'] = pd.to_datetime(df.Date.astype(str)+' '+d.Time.astype(str))
d = d.set_index(pd.DatetimeIndex(d['log']))
d.drop(columns=['log'],inplace = True)
d = d.resample('D').sum()

In [7]:
d.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
log,,,,,,,
2006-12-16,1209.176,44.942,93552.53,5180.8,444.281631,797.0,5862.0
2006-12-17,3390.460,238.658,345725.32,14398.6,3466.958222,5153.0,19464.0
2006-12-18,2203.826,194.526,347373.64,9247.2,2475.000000,3790.0,20050.0
2006-12-19,1666.194,204.410,348479.01,7094.0,2313.000000,8417.0,15904.0
2006-12-20,2225.748,201.092,348923.61,9313.0,1440.000000,3939.0,20147.0


<div class="alert alert-block alert-info">
    <b>SECTION COMPREHENSION</b>
    
 1.25% of each attributes have missing values ('?'s and '0's). Unknown values have been replaced with NaNs and further imputed with values of the previous time period. Dtype is converted into floats for improved data comprehension and the data resampled to daily.

<b>
<b>
Word count: 42
</div>

## 1. Problem Description

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>
<br>
<br>
A dataset regarding electricity usage is provided for a household. 

Key metrics: 

- four years (2006 - 2010);
- recorded every minute;
- 2,075,259 records; and
- seven variables (active power, reactive power, voltage, global intensity, sub-metering 1, sub-metering 2, and sub-metering 3). 

<br>
Purpose: use historical data to predict future energy consumption rates. ARIMA model is used, and further corrected using alternative models e.g. random forests, and LSTMS. 
<br>
<br>
Benefits the study:

- billing projections for electricity companies; and 
- investment decisions for cost-saving outcomes of more environmentally friendly electricity options. 
<br>
<br>
Word count: 93

</div>

<br>
<div class="alert alert-success">
<h1 align="center"> 2. Data Comprehension </h1> <br>
<a id="data"></a>

In [ ]:
d.shape
d.head(8)
d.info()
d.describe()

<br>
<div class="alert alert-warning">
<b>1. ASSESSMENT (~100 words)</b> 

<p> Put your narrative or explanatory text in this cell (or additional cells as necessary) <p>

</div>

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>
<br>
<br>
The code in this second section will be clarified through visualisations in section 3.3 - Data analysis. 
<br>
<br>
Word count: 17

</div>

<br>
<div class="alert alert-success">
<h1 align="center"> 3. Data Preparation </h1> <br>
<a id="prep"></a>

In [8]:
# Please see previous data preparations in the overview section of this notebook.

<br>
<div class="alert alert-warning">
<b>1. ASSESSMENT (~100 words)</b> 

<p> Put your narrative or explanatory text in this cell (or additional cells as necessary) <p>

</div>

<br>
<div class="alert alert-success">
<h1 align="center"> 4. Fitting the Algorithms </h1> <br>
<a id="alg"></a>

## 1. Experimental Setup

In [ ]:
#spliting data into train/test
split = int(0.5 * len(d))
train, test = d.iloc[:split].reset_index(), d.iloc[split:].reset_index()

train.to_csv('train.csv',index=False, index_label=False)
test.to_csv('test.csv',index=False, index_label=False)

<br>
<div class="alert alert-warning">
<b>2. ASSESSMENT (~100 words)</b> 

<p> Put your narrative or explanatory text in this cell (or additional cells as necessary) <p>

</div>

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>
<br>
<br>
Initially, a training set of 10% was allocated. However, to improve performance of the model, training set will be set as 50%. In some cases, increased to 60% for comparison. 
<br>
<br>
Word count: 30

</div>

## 2. Naive forecast

In [ ]:
from sklearn.metrics import mean_squared_error
# Persistence model from the NB1
# data - assume a dailre resample and a weekly time frame for the forecasting
# persostance does not train a model so only the test data is needed here -> assue test_data
# the persistence model forecasts the value for Monday in week 1 as Monday in week 2

# get a data frame with the original columns and the shifted columns, remove the first 7 values (nans due to shift), called per_df
test_df = test 

cols = ['Global_active_power', 'Global_reactive_power','Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2','Sub_metering_3']
per_df = pd.DataFrame() 
rmse_vector = list()
for k in cols:
    temp_df = pd.concat([test_df[k].shift(7)[7:], test_df[k][7:]], axis=1)
    temp_df.columns = [k+'_'+'t', k+'_'+'t+7']
    per_df = pd.concat([per_df, temp_df], axis=1)
    l1 = temp_df.iloc[:,0].astype(float)
    l2 = temp_df.iloc[:,1].astype(float)
    rmse_vector.append(sqrt(mean_squared_error(l1,l2)))
evaluation_df = pd.DataFrame(columns = cols)
evaluation_df.loc[0] = rmse_vector
evaluation_df

<br>
<div class="alert alert-warning">
<b>3. ASSESSMENT (~100 words)</b> 

<p> Put your narrative or explanatory text in this cell (or additional cells as necessary) <p>

</div>

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>
<br>
<br>
Base model: persistence.
<br>
<br>
Multi-step prediction is set to create a weekly forecast, therefore utilising a t-7 value for t. The analysis focuses on 'Global_active_power' only. The RMSE of 753 kilowatts is relatively high to the mean global active consumption of 1500. 
<br>
<br>
Word count: 41

</div>

## 3. Data Analysis

In [ ]:
# show boxplot to understand range of values for each measure of energy
d.boxplot(return_type ='dict', figsize=(15,8))
plt.ylim(0,20000)
plt.xlabel('Power Consumption Type', fontsize=12)
plt.ylabel('Usage', fontsize=12)
plt.title('Energy Usage by Power Consumption Type', fontsize=20)
pyplot.show();

In [9]:
# Sub-metering 3 is notably more varied compared to energy consumption rates of the remaining power consumption types, with global_reactive power 
# showing least variation. It should also be noted that measures for voltage are much higher and is beyond the presentable range. 

In [ ]:
# convert dataset into series 
d_series = pd.DataFrame(data=d)

# create plot for seven attributes to see trend
d_series.plot(subplots=True, color='grey', figsize=(10,30))
pyplot.show()

In [ ]:
# because trend is still not clear, possibly seeing total on a monthly basis will help identify trend 
d2_series = d.resample('M').sum()
d2_series.plot(subplots=True, legend=True, color='grey', figsize=(10,30))

In [12]:
# After converting the dataset into monthly basis, the trends appear to be clearer in all energy consumption types except for voltage. 
# Most consumption types show a fall in usage during the mid-year and peaks when introduced to a new month. 
# However, the trend or seasonality does not appear too apparent throughout and therefore can be considered as residual. 

In [ ]:
lag_plot(d2_series['Global_active_power'], c='grey')
plt.title('Global Active Power Lag Plot')
pyplot.show()
lag_plot(d2_series['Global_reactive_power'],c='grey')
pyplot.show()
lag_plot(d2_series['Voltage'],c='grey')
pyplot.show()
lag_plot(d2_series['Global_intensity'],c='grey')
pyplot.show()
lag_plot(d2_series['Sub_metering_1'],c='grey')
pyplot.show()
lag_plot(d2_series['Sub_metering_2'],c='grey')
pyplot.show()
lag_plot(d2_series['Sub_metering_3'],c='grey')
pyplot.show()

# edit to make loop and add in title

In [10]:
# The plot shows how a one-step time lag correlates to the present measure. The stronger the correlation (1), the more appropriate the dataset is 
# for ARIMA usage. Measures with the highest correlation include global active power, global reactive power, global intensity, and sub-metering 3. 

In [ ]:
# trend becomes more apparent when values assessed on a monthly basis, therefore will be set for the analysis. To double check monthly trend, a graph plotting yearly measures will be created. 
def plot_series(data):
    pyplot.figure(2, figsize=(15,10))
    groups = data.groupby(pd.Grouper(freq='A'))
    years = pd.DataFrame()
    i=1
    n_groups = len(groups)
    for name, group in groups: 
        pyplot.subplot((n_groups*100) + 10 + i)
        i += 1 
        pyplot.plot(group, color='grey')
    pyplot.xticks(rotation=90)
    return pyplot.show()

plot_series(d2_series['Global_active_power'])
plot_series(d2_series['Global_reactive_power'])
plot_series(d2_series['Voltage'])
plot_series(d2_series['Global_intensity'])
plot_series(d2_series['Sub_metering_1'])
plot_series(d2_series['Sub_metering_2'])
plot_series(d2_series['Sub_metering_3'])


In [11]:
#Note that daily analysis was converted into monthly to increase comprehension of seasonality. 

In [ ]:
d2_series['Global_active_power'].plot(kind='kde', color='grey')
pyplot.show()
d2_series['Global_reactive_power'].plot(kind='kde', color='grey')
pyplot.show()
d2_series['Voltage'].plot(kind='kde', color='grey')
pyplot.show()
d2_series['Global_intensity'].plot(kind='kde', color='grey')
pyplot.show()
d2_series['Sub_metering_1'].plot(kind='kde', color='grey')
pyplot.show()
d2_series['Sub_metering_2'].plot(kind='kde', color='grey')
pyplot.show()
d2_series['Sub_metering_3'].plot(kind='kde', color='grey')
pyplot.show()

<br>
<div class="alert alert-warning">
<b>4. ASSESSMENT (~200 words)</b> 

<p> Put your narrative or explanatory text in this cell (or additional cells as necessary) <p>

</div>

<div class="alert alert-block alert-info">
    
<b>GRAPH COMPREHENSION</b>
Note that daily analysis was converted into monthly to increase comprehension of seasonality. 
It was found that global active power is the most suitable electricity measure to be used for the time series analysis for the following reasons: <br>
<br>
- enough range;<br>
- high correlation in lag plot; and <br>
- completeness. <br>
<br>
Seasonality will be considered, however was inconsistent therefore does not have strong impact.<br>


Word count: 63

## 4. Linear Models

### 4.1 Check Stationarity

In [ ]:
# create and summarize stationary version of time series
from statsmodels.tsa.stattools import adfuller

def adfuller_test(data):
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    testing = adfuller(data, autolag='AIC')
    output = pd.Series(testing[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in testing[4].items():
        output['Critical Value (%s)'%key] = value
    print (output)

#apply adf test on the series
result1 = adfuller_test(d['Global_active_power'])
print('')
result2 = adfuller_test(d['Global_reactive_power'])
print('')
result3 = adfuller_test(d['Voltage'])
print('')
result4 = adfuller_test(d['Global_intensity'])
print('')
result5 = adfuller_test(d['Sub_metering_1'])
print('')
result6 = adfuller_test(d['Sub_metering_2'])
print('')
result7 = adfuller_test(d['Sub_metering_3'])

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>

The augmented Dickey-Fuller test was run on all univariate time series to check stationarity. Due to a value of less than 1%, null hypothesis was rejected and time series deemed stationary. 
<br>
<br>
Word count: 31
</div>

### 4.2 Detrend by Differencing

In [ ]:
from pandas import Series 
from matplotlib import pyplot
import pylab

# Set series1 for the Global active power variable
df = pd.read_csv('train.csv',header=0);
df['log'] = pd.to_datetime(df['log'].astype(str))
series1 = pd.Series(df['Global_active_power'].values , index=df['log'])


# create a function to difference the series
def difference(dataset, interval):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

X = series1.values
X = X.astype('float32')

# difference the data by calling the function
days_in_week = 7
stationary1 = difference(X, days_in_week) 
stationary1.index = series1.index[days_in_week:] 

# save as csv
stationary1.to_csv('stationary1.csv') 

# plot
pyplot.figure(5,figsize=(10, 8))
stationary1.plot()
pyplot.show(5)

### 4.3 Manually Configure ARIMA

In [ ]:
# ACF and PACF plots of time series
from statsmodels.graphics.tsaplots import plot_acf 
from statsmodels.graphics.tsaplots import plot_pacf 
#stationary 1

pyplot.figure(6,figsize=(15,10)) 
pyplot.plot(211)
pylab.ylim([-0.5,1])
plot_acf(stationary1, ax=pyplot.gca()) 

pyplot.figure(7,figsize=(15,10)) 
pyplot.plot(212)
pylab.ylim([-0.5,1])
plot_pacf(stationary1, ax=pyplot.gca())


<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>

In ACF and PACF, identifying lags will help determine ARIMA parameters. ARIMA is set at (3, 0, 1). The parameters result in an RMSE of 696, superior than in persistence model of RMSE 818. Building a better model may improve results. 

<br>
<br>
Word count: 41 
</div>

### 4.4 Running the ARIMA model

In [ ]:
# evaluate manually configured ARIMA model
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
from math import sqrt

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

# Set series1 for the Global active power variable
df = pd.read_csv('train.csv',header=0);
df['log'] = pd.to_datetime(df['log'].astype(str))
series1 = pd.Series(df['Global_active_power'].values , index=df['log'])

# prepare data
X = series1.values
X = X.astype('float32')
train_size = int(len(X) * 0.50)
train, test = X[0:train_size], X[train_size:]

# walk-forward validation
history = [x for x in train]
predictions = list()
for i in range(len(test)):
    
    # difference data
    days_in_week = 7
    diff = difference(history, days_in_week)
    
    # predict
    model = ARIMA(diff, order=(3,0,1)) #using manually selected parameters
    model_fit = model.fit(trend='nc', disp=0)
    yhat = model_fit.forecast()[0]
    yhat = inverse_difference(history, yhat, days_in_week)
    predictions.append(yhat)
    
    # observation
    obs = test[i]
    history.append(obs)
    print('>Predicted=%.3f, Expected=%3.f' % (yhat, obs))
    
# report performance
rmse = sqrt(mean_squared_error(test, predictions))
print('RMSE: %.3f' % rmse)

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>

Running the ARIMA (3, 0, 1) model results in an RMSE of 683, which is better than the persistence RMSE of 752. This is a great start, but we may be able to improve the result by building a better model.
<br>
<br>
Word count: 41 
</div>

### 4.5 Review the residual errors

In [ ]:
# summarize ARIMA forecast residuals
from pandas import DataFrame
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot

# prepare data
X = series1.values
X = X.astype('float32')
train_size = int(len(X) * 0.50)
train, test = X[0:train_size], X[train_size:]

# walk-forward validation
history = [x for x in train]
predictions = list()


for i in range(len(test)):
    # difference data
    days_in_weeks = 7
    diff = difference(history, days_in_weeks)
    
    # predict
    model = ARIMA(diff, order=(3,0,1))
    model_fit = model.fit(trend='nc', disp=0)
    yhat = model_fit.forecast()[0]
    yhat = inverse_difference(history, yhat, days_in_weeks)
    predictions.append(yhat)

    # observation
    obs = test[i]
    history.append(obs)

# errors
residuals = [test[i]-predictions[i] for i in range(len(test))]
residuals = DataFrame(residuals)
print(residuals.describe())


In [ ]:
# plot
pyplot.figure(8,figsize=(10, 8))
pyplot.subplot(211)
residuals.hist(ax=pyplot.gca())
pyplot.subplot(212)
residuals.plot(kind='kde', ax=pyplot.gca())
pyplot.show();

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>

In the second graph above, it shows the density plot of the residual error values. It suggests that the errors are Gaussian, however, it may not be centered on zero.
<br>
<br>
Word count: 30 
</div>

### 4.6 Grid search ARIMA model hyperparameters

In [ ]:
# grid search ARIMA parameters for time series
import warnings
from math import sqrt
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

def evaluate_arima_model(X, arima_order):
    train_size = int(len(X) * 0.50)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    predictions = list()
    for i in range(len(test)):
        days_in_weeks = 7
        diff = difference(history, days_in_weeks)
        model = ARIMA(diff, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        yhat = inverse_difference(history, yhat, days_in_weeks)
        predictions.append(yhat)
        history.append(test[i])
    # calculate out of sample error
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(dataset, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except:
                    continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

# evaluate parameters
p_values = [0,1, 3, 5]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series1.values, p_values, d_values, q_values)

### 4.7 Review residual errors for final model 

In [ ]:
# summarize ARIMA forecast residuals
from pandas import DataFrame
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot

def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

# prepare data
X = series1.values
X = X.astype('float32')
train_size = int(len(X) * 0.50)
train, test = X[0:train_size], X[train_size:]

# walk-forward validation
history = [x for x in train]
predictions = list()


for i in range(len(test)):
    # difference data
    days_in_weeks = 7
    diff = difference(history, days_in_weeks)
    
    # predict
    model = ARIMA(diff, order=(3,0,2))
    model_fit = model.fit(trend='nc', disp=0)
    yhat = model_fit.forecast()[0]
    yhat = inverse_difference(history, yhat, days_in_weeks)
    predictions.append(yhat)

    # observation
    obs = test[i]
    history.append(obs)

# errors
residuals = [test[i]-predictions[i] for i in range(len(test))]
residuals = DataFrame(residuals)
print(residuals.describe())

In [ ]:
# plot
pyplot.figure(8,figsize=(10, 8))
pyplot.subplot(211)
residuals.hist(ax=pyplot.gca())
pyplot.subplot(212)
residuals.plot(kind='kde', ax=pyplot.gca())
pyplot.show();

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>

Gridsearch suggests ARIMA (3, 0, 2). Compared to ARIMA (3, 0, 1), distribution is Gaussian and less bumpy. Mean remains non-zero but closer to 0. 

<br>
<br>
Word count: 25
</div>

### 4.8 Finalise model and make prediction

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from scipy.stats import boxcox
import numpy

# difference data
weeks_in_days = 7
diff = difference(X, weeks_in_days)

# fit model
model = ARIMA(diff, order=(3,0,2))
model_fit = model.fit(trend='nc', disp=0)

# save model
model_fit.save('model.pkl')

In [ ]:
# load finalized model and make a prediction
# load finalized model and make a prediction

weeks_in_days = 7
model_fit = ARIMAResults.load('model.pkl') 
yhat = float(model_fit.forecast()[0])
yhat = inverse_difference(series1.values, yhat, weeks_in_days)
print('Predicted: %.3f' % yhat)

### 4.9 Final model on validation set

In [ ]:
# load and evaluate the finalized model on the validation dataset

# load and prepare datasets
history = [x for x in X]
days_in_weeks = 7

# load data
df = pd.read_csv('train.csv',header=0);
df['log'] = pd.to_datetime(df['log'].astype(str))
train = pd.Series(df['Global_active_power'].values , index=df['log'])


# load data
# Set series1 for the Global active power variable
df = pd.read_csv('test.csv',header=0);
df['log'] = pd.to_datetime(df['log'].astype(str))
validation = pd.Series(df['Global_active_power'].values , index=df['log'])
prediction = validation #creat dummy series to hold prediction data

x = train.values.astype('float32')
y = validation.values.astype('float32')

# load model
model_fit = ARIMAResults.load('model.pkl')

# make first prediction
predictions = list()
yhat = float(model_fit.forecast()[0])
yhat = inverse_difference(history, yhat, days_in_weeks)
predictions.append(yhat)
history.append(y[0])
print('>Predicted=%.3f, Expected=%3.f' % (yhat, y[0]))

# rolling forecasts
for i in range(1, len(y)):
    # difference data
    days_in_weeks = 7
    diff = difference(history, days_in_weeks)
    
    # predict
    model = ARIMA(diff, order=(3,0,2))
    model_fit = model.fit(trend='nc', disp=0)
    yhat = model_fit.forecast()[0]
    yhat = inverse_difference(history, yhat, days_in_weeks)
    predictions.append(yhat)
    
    # observation
    obs = y[i]
    history.append(obs)
    
    prediction[i] = yhat    
    print('>Predicted=%.3f, Expected=%3.f' % (yhat, obs))

# report performance
rmse = sqrt(mean_squared_error(y, predictions))
print('RMSE: %.3f' % rmse)


In [ ]:
pyplot.figure(9,figsize=(10, 8))
train.plot(color='blue',label='train')
validation.plot(color='green',label='validation')
prediction.plot(color='red',label='prediction')
txt="The train, test and predictive time series comparison"
pyplot.figtext(0.5, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=12)
pyplot.legend(loc='upper left')
pyplot.show()

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>

One observation is that the RMSE for testing set is higher than the training set. We can vizualise this results by plotting the train, test and predictive time series together on the same graph.
<br>
<br>
Word count: 34
</div>

### 4.10 GARCH model
- motivations: GARCH - allows the method to support changes in the time dependent volatility, such as increasing and decreasing volatility in the same series

In [ ]:
from arch import arch_model
# Working with data
values = d.values

# integer encode direction
encoder = LabelEncoder()
values[:,6] = encoder.fit_transform(values[:,6])

# ensure all data is float
values = values.astype('float64')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
d1 = pd.DataFrame(scaler.fit_transform(values))
d1 = d1.set_index(d.index)
d1.columns = [i for i in d.columns]

#d1.drop([], )

In [ ]:
def evaluate_modelgarch(residuals, st_residuals, lags=50):
    results = {
        'LM_pvalue': None,
        'F_pvalue': None,
        'SW_pvalue': None,
        'AIC': None,
        'params': {'p': None, 'q': None}
    }
    arch_test = het_arch(residuals, maxlag=lags)
    shap_test = shapiro(st_residuals)
    # We want falsey values for each of these hypothesis tests
    results['LM_pvalue'] = [arch_test[1], arch_test[1] < .05]
    results['F_pvalue'] = [arch_test[3], arch_test[3] < .05]
    results['SW_pvalue'] = [shap_test[1], shap_test[1] < .05]
    return results


In [ ]:
def gridsearchgarch(data, p_rng, q_rng):
    top_score, top_results = float('inf'), None
    top_models = []
    for p in p_rng:
        for q in q_rng:
            try:
                model = arch_model(data, vol='GARCH', p=p, q=q, dist='normal')
                model_fit = model.fit(disp='off')
                resid = model_fit.resid
                st_resid = np.divide(resid, model_fit.conditional_volatility)
                results = evaluate_modelgarch(resid, st_resid)
                results['AIC'] = model_fit.aic
                results['params']['p'] = p
                results['params']['q'] = q
                if results['AIC'] < top_score: 
                    top_score = results['AIC']
                    top_results = results
                elif results['LM_pvalue'][1] is False:
                    top_models.append(results)
            except:
                continue
    top_models.append(top_results)
    return top_models


In [ ]:
def ts_plot(residuals, stan_residuals, lags=50):
    residuals.plot(title='GARCH Residuals', figsize=(15, 10))
    plt.show()
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
    ax[0].set_title('GARCH Standardized Residuals KDE')
    ax[1].set_title('GARCH Standardized Resduals Probability Plot')    
    residuals.plot(kind='kde', ax=ax[0])
    probplot(stan_residuals, dist='norm', plot=ax[1])
    plt.show()
    acf = plot_acf(stan_residuals, lags=lags)
    pacf = plot_pacf(stan_residuals, lags=lags)
    acf.suptitle('GARCH Model Standardized Residual Autocorrelation', fontsize=20)
    acf.set_figheight(5)
    acf.set_figwidth(15)
    pacf.set_figheight(5)
    pacf.set_figwidth(15)
    plt.show()


In [ ]:
def garchmodel(data, splits, series_column):
    plt.plot(d1[series_column])
    plt.title(series_column)
    plt.show()
    # Spliting the data
    split = int(splits * len(data)) #Note: change the split for the faster model training
    train, test = data[series_column][:split], data[series_column][split:]
    
    #do gridsearch
    p_rng = range(0,10)
    q_rng = range(0,10)
    top_models = gridsearchgarch(train, p_rng, q_rng)
    top_models

    # design garch
    garch = arch_model(train, vol='GARCH', p=top_models[0]['params']['p'], q=top_models[0]['params']['q'], dist='normal')
    fgarch = garch.fit(disp='off') 

    resid = fgarch.resid
    st_resid = np.divide(resid, fgarch.conditional_volatility)

    #model results
    print(fgarch.summary())
    arch_test = het_arch(resid, maxlag=50)
    shapiro_test = shapiro(st_resid)
    print(f'Lagrange mulitplier p-value: {arch_test[1]}')
    print(f'F test p-value: {arch_test[3]}')
    print(f'Shapiro-Wilks p-value: {shapiro_test[1]}')
    ts_plot(resid, st_resid)
  

In [ ]:
#in the 3 hours before the submition the problem with arch_model from the package arch occured
#due to the time constrain we could not deal with the problem
#besides the results were already achived

#to check the results obtained go to GARCH-result Jupyter Nootbook

for i in d.columns[:7]:
    garchmodel(d1, 0.5, i )

In [ ]:
#Note:
# - omega -
# - alfa - 22.1% of the pervious periods volatility will be passed to the next day.
# - beta <1 no leading to a postive feedback loop for small shocks that can create runaway volatility.
# if alfa + beta =1, model has persistent volatility and we might want to look at other models like IGARCH.

# - p value is <0.05, so coeffisient has an effect

# check if the model has model has captured the conditional heteroskedasticity of our time series:
# - Lagrange multiplier test - residuals not exibit heterostedasity
# - F-test - Data  normaly distributed (for n<2000)
# - Sharpiro-Wilks - data normaly distributed


<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>

The GARCH models have passed our Engel ARCH test and the standardized residuals appear approximately normal. However, it is has to capture the voliatility and all these model seak more  precice consideration
<br>
<br>
Word count: 32
</div>

<br>
<div class="alert alert-warning">
<b>5. ASSESSMENT (~200 words) ---> For this section, please see the blue boxes above! </b> 

<p> Put your narrative or explanatory text in this cell (or additional cells as necessary) <p>

</div>

## 5. Supervised Learning formulation

### Feature engineering
***supevised setup, conceptualisation***

In [ ]:
# add the dail average temperature (i think it is farenheit?) to the data
climate = pd.read_csv("climate_data.csv")
climate = climate.loc[climate.STATION == 'FR000007150'].iloc[15:1472,:]
d['climate'] = list(climate.iloc[:,3])
climate.shape, d.shape
# add weekend indicator
days = list(d.index.weekday)
weekend = [1 if (j==5 or j ==6) else 0 for j in days]
d['weekend'] = weekend

In [ ]:
len(climate.iloc[:,3])

In [ ]:
nlist = list()
for col in climate.columns:
    nlist.append(len([i for i in climate[col].isnull() if i==True])/climate.shape[0])
labels = [i for i in climate.columns]
print(nlist)
print(labels)

In [ ]:
#Important for technical understanding!

# This is a "multivariate multi-step time series forecasting problem" in a supervised framework

# Using the lag method to create supervised learning dataset -> here 7 time lags are used
# Therefore we are in t0 and we predict the values t+1...t+7 (like Mo-Sun) based on t-7...t0 and the corresponding # moving window stats
# Incorporated time stamp features (seasonality, weekend etc..)
# Incorporated Paris weather data (temperatures) because the the measurements were taken in a place 7 miles from # Paris (temperature might add variation that can help explain the consumption, e.g. cold and rainy day vs. sunny and # warm)


### Re-framing the problem to make it a supervised one ###
#The issue is that we have multiple inputs as usual) but also multiple outputs (Global active power in future 7 # periods)!  Normal models cannot learn that, they only yield one output value. We need to re-frame the problem:

# TWO APPROACHES:
# 1) Direct multi-step approach: Build 7 classifiers, each classifies one day (all 7 based on the same data)
# 2) Recursive multi-step approach: One-step model predicts based on window and previous forecast step, 
# e.g. today == Sunday, preciction(Tuesday) is a function of window history, window statistics AND # prediction(Monday)

#  The second seems more promising. We implement the direct approach first and see whether we have time left.


cols = ['Global_active_power', 'Global_reactive_power','Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2','Sub_metering_3','climate','weekend']
df_sup_all = pd.DataFrame()
for k in cols:
        temps = d[k]
        df_sup = pd.concat([temps.shift(7)-temps.shift(14),temps.shift(14),temps.shift(13),temps.shift(12),temps.shift(11),temps.shift(10),temps.shift(9),temps.shift(8),temps.shift(7),temps.shift(6),temps.shift(5),temps.shift(4),temps.shift(3), temps.shift(2), temps.shift(1), temps], axis=1)
        df_sup.columns = [k+'_'+'m_change',k+'_'+'t-7', k+'_'+'t-6' ,k+'_'+'t-5' ,k+'_'+'t-4' ,k+'_'+'t-3' ,k+'_'+'t-2' ,k+'_'+'t-1' ,k+'_'+ 't-0', k+'_'+'t+1', k+'_'+'t+2', k+'_'+'t+3', k+'_'+'t+4', k+'_'+'t+5', k+'_'+'t+6',k+'_'+ 't+7']
        m_mean = np.mean(df_sup.iloc[:,:-1], axis = 1)
        m_min = np.min(df_sup.iloc[:,:-1], axis = 1)
        m_max = np.max(df_sup.iloc[:,:-1], axis = 1)
        m_std = np.std(df_sup.iloc[:,:-1], axis = 1)
        df_sup.insert(loc=0, column = k+'_'+'m_mean', value = m_mean)
        df_sup.insert(loc=0, column = k+'_'+'m_min', value = m_min)
        df_sup.insert(loc=0, column = k+'_'+'m_max', value = m_max)
        df_sup.insert(loc=0, column = k+'_'+'m_std', value = m_std)
        df_sup_all = pd.concat([df_sup_all,df_sup], axis = 1)
        
# delete the first 14 rows due to the time lags! 
df_sup_all = df_sup_all.iloc[14:,:]

# note: the test, train split from earlier must be incorporated again, since this is the full dataset

In [ ]:
# 1443 days to forecast (t+1...t+7 or equivalent to Monday to Sunday), 140 features generated plus some climate features
df_sup_all.shape

In [ ]:
# the matriix contains 4 sliding window statistics and the 7 days previous days, and the 7 follwoing days that we need to predict (7 outcomes)
df_sup_all.head(3)

### Supervised modelling, base classifiers
***Direct approach to multistep time series***

In [ ]:
# Create a direct modelling approach
data = df_sup_all.iloc[:,13:20]
temp_df = pd.concat([df_sup_all.iloc[:,0:13], df_sup_all.iloc[:,20:33],df_sup_all.iloc[:,40:53],df_sup_all.iloc[:,60:73],df_sup_all.iloc[:,80:93],df_sup_all.iloc[:,100:113],df_sup_all.iloc[:,120:133],df_sup_all.iloc[:,140:153],df_sup_all.iloc[:,160:173]],axis = 1)
df_direct = pd.concat([temp_df,data],axis = 1)
df_direct.shape
df_direct_fp = df_direct

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df_direct = scaler.fit_transform(df_direct)
df_direct = pd.DataFrame(df_direct)
n_train = int(0.5 * len(d))
train_X = df_direct.iloc[:n_train,:117]
train_y = df_direct.iloc[:n_train,117:]
test_X = df_direct.iloc[n_train:,:117]
test_y = df_direct.iloc[n_train:,117:]
train_X.shape, train_y.shape, test_X.shape, test_y.shape

In [ ]:
# copy for the polynomial regression only
df_direct_fp.shape

<div class="alert alert-info">
    <b>SECTION COMPREHENSION</b>
    <br>
Data is normalised by using a scaler. Features are in range 0 to 1 from this point and used in train and test subsets. 
<br>
<br>
Word count: 24

### Linear Regression

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
reg_t_1 = LinearRegression().fit(train_X, train_y.iloc[:,0])
reg_t_2 = LinearRegression().fit(train_X, train_y.iloc[:,1])
reg_t_3 = LinearRegression().fit(train_X, train_y.iloc[:,2])
reg_t_4 = LinearRegression().fit(train_X, train_y.iloc[:,3])
reg_t_5 = LinearRegression().fit(train_X, train_y.iloc[:,4])
reg_t_6 = LinearRegression().fit(train_X, train_y.iloc[:,5])
reg_t_7 = LinearRegression().fit(train_X, train_y.iloc[:,6])

reg_t_1_pred = pd.Series(reg_t_1.predict(test_X))
reg_t_2_pred = pd.Series(reg_t_2.predict(test_X))
reg_t_3_pred = pd.Series(reg_t_3.predict(test_X))
reg_t_4_pred = pd.Series(reg_t_4.predict(test_X))
reg_t_5_pred = pd.Series(reg_t_5.predict(test_X))
reg_t_6_pred = pd.Series(reg_t_6.predict(test_X))
reg_t_7_pred = pd.Series(reg_t_7.predict(test_X))

#from sklearn.metrics import mean_squared_error
#rmse = sqrt(mean_squared_error(test_y.iloc[:,0],reg_t_1_pred))
#print(rmse)


# invert scaling:
# matrix of predicted values with shape of test_y
yhat = pd.concat([reg_t_1_pred,reg_t_2_pred,reg_t_3_pred,reg_t_4_pred,reg_t_5_pred,reg_t_6_pred,reg_t_7_pred],axis = 1)

inv_yhat = np.concatenate((test_X ,yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = pd.DataFrame(inv_yhat).iloc[:,117:]

inv_y = np.concatenate((test_X, test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = pd.DataFrame(inv_y).iloc[:,117:]

# calculate RMSE, combine
temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k])))
rmse = np.sum(temp)/7    
print('Overall Test RMSE: %.3f' % rmse)

inter = [x for x in range(inv_y.shape[0])]
plt.figure(figsize = (20,8))
plt.plot(inter, inv_y.iloc[:inv_y.shape[0],0], marker='.', label="actual")
plt.plot(inter, inv_yhat.iloc[:inv_y.shape[0],0], 'r', label="prediction")
plt.title('Predictions and actual values for t+1 step in multi-step 7 days')
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

<div class="alert alert-info">
    <b>MODEL COMPREHENSION</b>
    <br>
Linear regression is applied with direct approach, however resulting in high levels of RMSE. It is inferred that around time step 400, variation of predictions drastically increase. 

<br>
<br>
Word count: 27

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
clf_d = DecisionTreeRegressor()

tree_t_1 = clf_d.fit(train_X, train_y.iloc[:,0])
tree_t_2 = clf_d.fit(train_X, train_y.iloc[:,1])
tree_t_3 = clf_d.fit(train_X, train_y.iloc[:,2])
tree_t_4 = clf_d.fit(train_X, train_y.iloc[:,3])
tree_t_5 = clf_d.fit(train_X, train_y.iloc[:,4])
tree_t_6 = clf_d.fit(train_X, train_y.iloc[:,5])
tree_t_7 = clf_d.fit(train_X, train_y.iloc[:,6])

tree_t_1_pred = pd.Series(tree_t_1.predict(test_X))
tree_t_3_pred = pd.Series(tree_t_3.predict(test_X))
tree_t_4_pred = pd.Series(tree_t_4.predict(test_X))
tree_t_5_pred = pd.Series(tree_t_5.predict(test_X))
tree_t_6_pred = pd.Series(tree_t_6.predict(test_X))
tree_t_7_pred = pd.Series(tree_t_7.predict(test_X))


#rmse_tree = sqrt(mean_squared_error(test_y.iloc[:,0],tree_t_1_pred))
#print(rmse_tree)

tree_t_1_pred = pd.Series(tree_t_1.predict(test_X))
tree_t_2_pred = pd.Series(tree_t_2.predict(test_X))
tree_t_3_pred = pd.Series(tree_t_3.predict(test_X))
tree_t_4_pred = pd.Series(tree_t_4.predict(test_X))
tree_t_5_pred = pd.Series(tree_t_5.predict(test_X))
tree_t_6_pred = pd.Series(tree_t_6.predict(test_X))
tree_t_7_pred = pd.Series(tree_t_7.predict(test_X))

yhat_d = pd.concat([tree_t_1_pred,tree_t_2_pred,tree_t_3_pred,tree_t_4_pred,tree_t_5_pred,tree_t_6_pred,tree_t_7_pred],axis = 1)

inv_yhat_d = np.concatenate((test_X ,yhat_d), axis=1)
inv_yhat_d = scaler.inverse_transform(inv_yhat_d)
inv_yhat_d = pd.DataFrame(inv_yhat_d).iloc[:,117:]

inv_y_d = np.concatenate((test_X, test_y), axis=1)
inv_y_d = scaler.inverse_transform(inv_y_d)
inv_y_d = pd.DataFrame(inv_y_d).iloc[:,117:]

# RMSE and combine
temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y_d.iloc[:,k], inv_yhat_d.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y_d.iloc[:,k], inv_yhat_d.iloc[:,k])))
rmse = np.sum(temp)/7    
print('Overall Test RMSE: %.3f' % rmse)

In [ ]:
inter = [x for x in range(inv_y_d.shape[0])]
plt.figure(figsize = (20,8))
plt.plot(inter, inv_y.iloc[:inv_y.shape[0],0], marker='.', label="actual")
plt.plot(inter, inv_yhat_d.iloc[:inv_y.shape[0],0], 'r', label="prediction")
plt.title('Predictions and actual values for t+1 step in multi-step 7 days')
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

<div class="alert alert-info">
    <b>MODEL COMPREHENSION</b>
    <br>
A non-linear decision tree is deployed to test a second supervised algorithm. An RMSE of 957 shows improved performance compared to linear regression, with less variation. 
<br>
<br>
Word count: 26

### Polynomial

In [ ]:
# Polynomial transformation before splitting into train and test
from sklearn.preprocessing import PolynomialFeatures
polynomial_features = PolynomialFeatures(degree=2)
train_temp = df_direct_fp.iloc[:,:117]
test_temp = df_direct_fp.iloc[:,117:]
train_temp_poly = pd.DataFrame(polynomial_features.fit_transform(train_temp))
test_temp.reset_index(drop=True, inplace=True)
train_temp_poly.reset_index(drop=True, inplace=True)
df_direct_poly = pd.concat([train_temp_poly,test_temp],axis = 1)
df_direct_poly.shape
scaler = MinMaxScaler(feature_range=(0, 1))
df_direct_poly = scaler.fit_transform(df_direct_poly)
df_direct_poly = pd.DataFrame(df_direct_poly)
n_train = int(0.5 * len(d))
train_X_p = df_direct_poly.iloc[:n_train,:7021]
train_y_p = df_direct_poly.iloc[:n_train,7021:]
test_X_p = df_direct_poly.iloc[n_train:,:7021]
test_y_p = df_direct_poly.iloc[n_train:,7021:]
train_X_p.shape, train_y_p.shape, test_X_p.shape, test_y_p.shape

In [ ]:
# Linear regression with polynominal features

rpoly_t_1 = LinearRegression().fit(train_X_p, train_y_p.iloc[:,0])
rpoly_t_2 = LinearRegression().fit(train_X_p, train_y_p.iloc[:,1])
rpoly_t_3 = LinearRegression().fit(train_X_p, train_y_p.iloc[:,2])
rpoly_t_4 = LinearRegression().fit(train_X_p, train_y_p.iloc[:,3])
rpoly_t_5 = LinearRegression().fit(train_X_p, train_y_p.iloc[:,4])
rpoly_t_6 = LinearRegression().fit(train_X_p, train_y_p.iloc[:,5])
rpoly_t_7 = LinearRegression().fit(train_X_p, train_y_p.iloc[:,6])

#from sklearn.metrics import mean_squared_error
#rmse = sqrt(mean_squared_error(test_y_p.iloc[:,0],rpoly_t_1_pred))
#print(rmse)

rpoly_t_1_pred = pd.Series(rpoly_t_1.predict(test_X_p))
rpoly_t_2_pred = pd.Series(rpoly_t_2.predict(test_X_p))
rpoly_t_3_pred = pd.Series(rpoly_t_3.predict(test_X_p))
rpoly_t_4_pred = pd.Series(rpoly_t_4.predict(test_X_p))
rpoly_t_5_pred = pd.Series(rpoly_t_5.predict(test_X_p))
rpoly_t_6_pred = pd.Series(rpoly_t_6.predict(test_X_p))
rpoly_t_7_pred = pd.Series(rpoly_t_7.predict(test_X_p))

yhat = pd.concat([rpoly_t_1_pred,rpoly_t_2_pred,rpoly_t_3_pred,rpoly_t_4_pred,rpoly_t_5_pred,rpoly_t_6_pred,rpoly_t_7_pred],axis = 1)

inv_yhat_p = np.concatenate((test_X_p ,yhat), axis=1)
inv_yhat_p = scaler.inverse_transform(inv_yhat_p)
inv_yhat_p = pd.DataFrame(inv_yhat_p).iloc[:,7021:]

inv_y = np.concatenate((test_X_p, test_y_p), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = pd.DataFrame(inv_y).iloc[:,7021:]

# calculate RMSE, combine
temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat_p.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat_p.iloc[:,k])))
rmse = np.sum(temp)/7    
print('Overall Test RMSE: %.3f' % rmse)
# over-fitting -> rugularisation needed

In [ ]:
intertree = [x for x in range(inv_y_d.shape[0])]
plt.figure(figsize = (20,8))
plt.plot(intertree, inv_y.iloc[:inv_y.shape[0],0], marker='.', label="actual")
plt.plot(intertree, inv_yhat_p.iloc[:inv_y.shape[0],0], 'r', label="prediction")
plt.title('Predictions and actual values for t+1 step - multi-step 7 days')
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

In [ ]:
# Ridge regression with polynominal features
from sklearn.linear_model import Ridge
ridgereg_1 = Ridge(alpha = 1.0)
rpoly_t_1 = ridgereg_1.fit(train_X_p, train_y_p.iloc[:,0])
rpoly_t_2 = ridgereg_1.fit(train_X_p, train_y_p.iloc[:,1])
rpoly_t_3 = ridgereg_1.fit(train_X_p, train_y_p.iloc[:,2])
rpoly_t_4 = ridgereg_1.fit(train_X_p, train_y_p.iloc[:,3])
rpoly_t_5 = ridgereg_1.fit(train_X_p, train_y_p.iloc[:,4])
rpoly_t_6 = ridgereg_1.fit(train_X_p, train_y_p.iloc[:,5])
rpoly_t_7 = ridgereg_1.fit(train_X_p, train_y_p.iloc[:,6])

#from sklearn.metrics import mean_squared_error
#rmse = sqrt(mean_squared_error(test_y_p.iloc[:,0],rpoly_t_1_pred))
#print(rmse)

rpoly_t_1_pred = pd.Series(rpoly_t_1.predict(test_X_p))
rpoly_t_2_pred = pd.Series(rpoly_t_2.predict(test_X_p))
rpoly_t_3_pred = pd.Series(rpoly_t_3.predict(test_X_p))
rpoly_t_4_pred = pd.Series(rpoly_t_4.predict(test_X_p))
rpoly_t_5_pred = pd.Series(rpoly_t_5.predict(test_X_p))
rpoly_t_6_pred = pd.Series(rpoly_t_6.predict(test_X_p))
rpoly_t_7_pred = pd.Series(rpoly_t_7.predict(test_X_p))

yhat = pd.concat([rpoly_t_1_pred,rpoly_t_2_pred,rpoly_t_3_pred,rpoly_t_4_pred,rpoly_t_5_pred,rpoly_t_6_pred,rpoly_t_7_pred],axis = 1)

inv_yhat_p = np.concatenate((test_X_p ,yhat), axis=1)
inv_yhat_p = scaler.inverse_transform(inv_yhat_p)
inv_yhat_p = pd.DataFrame(inv_yhat_p).iloc[:,7021:]

inv_y = np.concatenate((test_X_p, test_y_p), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = pd.DataFrame(inv_y).iloc[:,7021:]

# calculate RMSE, combine
temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat_p.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat_p.iloc[:,k])))
rmse = np.sum(temp)/7    
print('Overall Test RMSE: %.3f' % rmse)

In [ ]:
# Ridge with alpha = 0.5
ridgereg_0_5 = Ridge(alpha = 0.5)
rpoly_t_1 = ridgereg_0_5.fit(train_X_p, train_y_p.iloc[:,0])
rpoly_t_2 = ridgereg_0_5.fit(train_X_p, train_y_p.iloc[:,1])
rpoly_t_3 = ridgereg_0_5.fit(train_X_p, train_y_p.iloc[:,2])
rpoly_t_4 = ridgereg_0_5.fit(train_X_p, train_y_p.iloc[:,3])
rpoly_t_5 = ridgereg_0_5.fit(train_X_p, train_y_p.iloc[:,4])
rpoly_t_6 = ridgereg_0_5.fit(train_X_p, train_y_p.iloc[:,5])
rpoly_t_7 = ridgereg_0_5.fit(train_X_p, train_y_p.iloc[:,6])

#from sklearn.metrics import mean_squared_error
#rmse = sqrt(mean_squared_error(test_y_p.iloc[:,0],rpoly_t_1_pred))
#print(rmse)

rpoly_t_1_pred = pd.Series(rpoly_t_1.predict(test_X_p))
rpoly_t_2_pred = pd.Series(rpoly_t_2.predict(test_X_p))
rpoly_t_3_pred = pd.Series(rpoly_t_3.predict(test_X_p))
rpoly_t_4_pred = pd.Series(rpoly_t_4.predict(test_X_p))
rpoly_t_5_pred = pd.Series(rpoly_t_5.predict(test_X_p))
rpoly_t_6_pred = pd.Series(rpoly_t_6.predict(test_X_p))
rpoly_t_7_pred = pd.Series(rpoly_t_7.predict(test_X_p))

yhat = pd.concat([rpoly_t_1_pred,rpoly_t_2_pred,rpoly_t_3_pred,rpoly_t_4_pred,rpoly_t_5_pred,rpoly_t_6_pred,rpoly_t_7_pred],axis = 1)

inv_yhat_p = np.concatenate((test_X_p ,yhat), axis=1)
inv_yhat_p = scaler.inverse_transform(inv_yhat_p)
inv_yhat_p = pd.DataFrame(inv_yhat_p).iloc[:,7021:]

inv_y = np.concatenate((test_X_p, test_y_p), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = pd.DataFrame(inv_y).iloc[:,7021:]

# calculate RMSE, combine
temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat_p.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat_p.iloc[:,k])))
rmse = np.sum(temp)/7    
print('Overall Test RMSE: %.3f' % rmse)

In [ ]:
# Ridge with alpha = 0.1
ridgereg_0_1 = Ridge(alpha = 0.1)
rpoly_t_1 = ridgereg_0_1.fit(train_X_p, train_y_p.iloc[:,0])
rpoly_t_2 = ridgereg_0_1.fit(train_X_p, train_y_p.iloc[:,1])
rpoly_t_3 = ridgereg_0_1.fit(train_X_p, train_y_p.iloc[:,2])
rpoly_t_4 = ridgereg_0_1.fit(train_X_p, train_y_p.iloc[:,3])
rpoly_t_5 = ridgereg_0_1.fit(train_X_p, train_y_p.iloc[:,4])
rpoly_t_6 = ridgereg_0_1.fit(train_X_p, train_y_p.iloc[:,5])
rpoly_t_7 = ridgereg_0_1.fit(train_X_p, train_y_p.iloc[:,6])

#from sklearn.metrics import mean_squared_error
#rmse = sqrt(mean_squared_error(test_y_p.iloc[:,0],rpoly_t_1_pred))
#print(rmse)

rpoly_t_1_pred = pd.Series(rpoly_t_1.predict(test_X_p))
rpoly_t_2_pred = pd.Series(rpoly_t_2.predict(test_X_p))
rpoly_t_3_pred = pd.Series(rpoly_t_3.predict(test_X_p))
rpoly_t_4_pred = pd.Series(rpoly_t_4.predict(test_X_p))
rpoly_t_5_pred = pd.Series(rpoly_t_5.predict(test_X_p))
rpoly_t_6_pred = pd.Series(rpoly_t_6.predict(test_X_p))
rpoly_t_7_pred = pd.Series(rpoly_t_7.predict(test_X_p))

yhat = pd.concat([rpoly_t_1_pred,rpoly_t_2_pred,rpoly_t_3_pred,rpoly_t_4_pred,rpoly_t_5_pred,rpoly_t_6_pred,rpoly_t_7_pred],axis = 1)

inv_yhat_p = np.concatenate((test_X_p ,yhat), axis=1)
inv_yhat_p = scaler.inverse_transform(inv_yhat_p)
inv_yhat_p = pd.DataFrame(inv_yhat_p).iloc[:,7021:]

inv_y = np.concatenate((test_X_p, test_y_p), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = pd.DataFrame(inv_y).iloc[:,7021:]

# calculate RMSE, combine
temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat_p.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat_p.iloc[:,k])))
rmse = np.sum(temp)/7    
print('Overall Test RMSE: %.3f' % rmse)

<div class="alert alert-info">
    <b>MODEL COMPREHENSION</b>
    <br>
To further improve accuracy, polynomial transformation of feature inputs to linear and ridge regression models is conducted. Performance improves slightly for linear however ridge (alpha = 1) predicts in a more precise manner. 

<br>
<br>
Word count: 33

<br>
<div class="alert alert-warning">
<b>6. ASSESSMENT (~200 words)</b> 

<p> Put your narrative or explanatory text in this cell (or additional cells as necessary) <p>

</div>

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>
<br>
<br>
This is a multi-step forecasting task. A new dataset with seven time steps prior and ahead. Additionally, rolling window statistics for each variable, weather approximation in Paris (close to where data was generated) and weekend indicators were created. Therefore, information for t-6 to t can be collected. Outcomes from t+1 to t+7 are written. A separate classifier will be created in the direct approach for each day of the week. Consequently, weekly predictions are made. RMSE scores will be averaged. 

<br>
<br>
Word count: 80

</div>

## 6. Ensemble Models

In [ ]:
# Create a direct modelling approach
data = df_sup_all.iloc[:,13:20]
temp_df = pd.concat([df_sup_all.iloc[:,0:13], df_sup_all.iloc[:,20:33],df_sup_all.iloc[:,40:53],df_sup_all.iloc[:,60:73],df_sup_all.iloc[:,80:93],df_sup_all.iloc[:,100:113],df_sup_all.iloc[:,120:133],df_sup_all.iloc[:,140:153],df_sup_all.iloc[:,160:173]],axis = 1)
df_direct = pd.concat([temp_df,data],axis = 1)
df_direct.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df_direct = scaler.fit_transform(df_direct)
df_direct = pd.DataFrame(df_direct)
n_train = int(0.5 * len(d))
train_X = df_direct.iloc[:n_train,:117]
train_y = df_direct.iloc[:n_train,117:]
test_X = df_direct.iloc[n_train:,:117]
test_y = df_direct.iloc[n_train:,117:]
train_X.shape, train_y.shape, test_X.shape, test_y.shape

In [ ]:
# RANDOM FORESTS MULTI-STEP 7 DAYS

from sklearn.ensemble import RandomForestRegressor

#classifiers = []
yhat = pd.DataFrame()
for k in range(7):
    model = RandomForestRegressor().fit(train_X, train_y.iloc[:,k])
    yhat['rf_'+str(k)] = model.predict(test_X)
    
inv_yhat = np.concatenate((test_X ,yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = pd.DataFrame(inv_yhat).iloc[:,117:]

inv_y = np.concatenate((test_X, test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = pd.DataFrame(inv_y).iloc[:,117:]

temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k])))
rmse = np.sum(temp)/7    
print('----------------------------')
print('Overall Test RMSE: %.3f' % rmse)

inter=[x for x in range(inv_y.shape[0])]
plt.figure(figsize = (10,7))
plt.plot(inter, inv_y.iloc[:inv_y.shape[0],0], marker='.', label="actual")
plt.plot(inter, inv_yhat.iloc[:inv_y.shape[0],0], 'r', label="prediction")
plt.title('Predictions and actual values for t+1 step - all test data')
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

In [ ]:
# GRADIENT BOOSTING

from sklearn.ensemble import GradientBoostingRegressor

yhat = pd.DataFrame()
for k in range(7):
    model = GradientBoostingRegressor().fit(train_X, train_y.iloc[:,k])
    yhat['rf_'+str(k)] = model.predict(test_X)
    
inv_yhat = np.concatenate((test_X ,yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = pd.DataFrame(inv_yhat).iloc[:,117:]

inv_y = np.concatenate((test_X, test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = pd.DataFrame(inv_y).iloc[:,117:]

gbm_inv_y_save = inv_y.iloc[:,-1]
gbm_inv_yhat_save = inv_yhat.iloc[:,-1]

temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k])))
rmse = np.sum(temp)/7    
print('----------------------------')
print('Overall Test RMSE: %.3f' % rmse)

inter=[x for x in range(inv_y.shape[0])]
plt.figure(figsize = (10,7))
plt.plot(inter, inv_y.iloc[:inv_y.shape[0],0], marker='.', label="actual")
plt.plot(inter, inv_yhat.iloc[:inv_y.shape[0],0], 'r', label="prediction")
plt.title('Predictions and actual values for t+1 step - all test data')
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

<br>
<div class="alert alert-warning">
<b>7. ASSESSMENT (~200 words)</b> 

<p> Put your narrative or explanatory text in this cell (or additional cells as necessary) <p>

</div>

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>
<br>
<br>
A bagging-based, random forest (RF), and boosting-based, gradient boosting (GBM) approach is chosen to deal with the instability of non-linear methods. Both perform much better than persistence model. GBM has highest performance (average RMSE of 522). It is still wrong by circa 1/3 of average daily energy consumption. Note that for both ensembles, RMSE increased from t+1 to t+7, which makes sense because it is further in the future. 

<br>
<br>
Word count: 69

</div>

## 7. Advanced Methods 

If you are confident, explore some of the more advanced methods that we covered in the course. 

For example, try and get a RNN working on the dataset you have chosen.

#### This section consists of serveral neural networks:<br>
- Model 1: Multivariate time series forecasting with NN
- Model 2: Multi-step forecasting of 7 days consumption ahead based on 1 time lag and all features available (see supervised framing section)
- Model 3: Multi-step forecasting of 7 days consumption ahead based on 7 time lag and all features available (see supervised framing section)

### Model 1:

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [ ]:
# Working with data
values = d.values

# integer encode direction
encoder = LabelEncoder()
values[:,6] = encoder.fit_transform(values[:,6])

# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

# frame as supervised learning
d1 = series_to_supervised(scaled, 1, 1)
d1 = d1[['var1(t-1)', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)', 'var5(t-1)','var6(t-1)', 'var7(t-1)', 'var8(t-1)', 'var8(t)', 'var9(t)', #use to predict
        'var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)', 'var6(t)', 'var7(t)', #need to predict
        'var9(t-1)']] #can be eliminated

# keep only the ones that need to be predicted
d1.drop(['var9(t-1)'], axis=1, inplace=True)


In [ ]:
def network_model(data, split_ratio, parameter_to_estemate):
    dict1 = dict(zip([i for i in d.columns], range(-7, 0)))
    i = dict1[parameter_to_estemate]
    # Spliting and spaping the data
    split = int(split_ratio * len(d)) #Note: change the split for the faster model training
    train, test = data[:split], data[split:]
    train_X, train_y = train.values[:, :-7], train.values[:, -7]
    test_X, test_y = test.values[:, :-7], test.values[:, -7]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    
    # design network
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    # make a prediction
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], 10)) #change to have same dimentions

    # invert scaling for forecast
    inv_yhat = np.concatenate((yhat, test_X[:, 2:]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((test_y, test_X[:, 2:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    # calculate RMSE
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    print(f'{parameter_to_estemate}, RMSE: {rmse}')
    
    plt.figure(figsize = (30,10))
    #train test loss
    plt.subplot(131)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()

    #Plot true labels
    plt.subplot(132)
    inter=[x for x in range(inv_y.shape[0])]
    plt.plot(inter, inv_y, marker='.', label="actual")
    plt.plot(inter, inv_yhat, 'r', label="prediction")
    plt.title('Predictions and actual values for t step')
    plt.ylabel('Global_active_power', size=15)
    plt.xlabel('Time step', size=15)
    plt.legend(fontsize=15)

    #Plot predicted labels with the decision boundaries
    plt.subplot(133)
    inter=[x for x in range(30)]
    plt.plot(inter, inv_y[:30], marker='.', label="actual")
    plt.plot(inter, inv_yhat[:30], 'r', label="prediction")
    plt.title('Predictions and actual values for t+1 step - 1 month')
    plt.ylabel('Global_active_power', size=15)
    plt.xlabel('Time step', size=15)
    plt.legend(fontsize=15)
    plt.show();


In [ ]:
for i in d.columns[:7]:
    network_model(d1, 0.6, i )


<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>

This model looks at multivariate time series forcasting with the LSTM. For better forecast model considerate climate and weekend variables.
<br>
<br>
Word count: 19
</div>

### Keras: Multivariate time series forecasting with LSTMs
### Models 2 and 3:

In [ ]:
# Two alternatives, model2 and model3
# Model 2 and 3 both predict multi-step although are based on different time lags

In [ ]:
# Model 2 implementation: 1 time lag, all features plus weather and weekend features, 
# input format [samples, timesteps, features] = [samples, 1, 9]
from sklearn.preprocessing import MinMaxScaler
data = df_sup_all.iloc[:,12:20]
temp_df = pd.concat([df_sup_all.iloc[:,32:33],df_sup_all.iloc[:,52:53],df_sup_all.iloc[:,72:73],df_sup_all.iloc[:,92:93],df_sup_all.iloc[:,112:113],df_sup_all.iloc[:,132:133],df_sup_all.iloc[:,152:153],df_sup_all.iloc[:,172:173]],axis = 1)
data = pd.concat([temp_df,data],axis = 1)
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)
data = pd.DataFrame(data)
n_train = int(0.5 * len(d))
train_X = data.iloc[:n_train,:9]
train_X = train_X.values.reshape(train_X.shape[0],1,train_X.shape[1])
train_y = data.iloc[:n_train,9:]
test_X = data.iloc[n_train:,:9]
test_X = test_X.values.reshape(test_X.shape[0],1,test_X.shape[1])
test_y = data.iloc[n_train:,9:]
train_X.shape, train_y.shape, test_X.shape, test_y.shape

In [ ]:
# lstm
# modelling choices

model = Sequential()
model.add(LSTM(200,activation='relu',return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(100,activation = 'relu',return_sequences=False))
model.add(Dense(7))
model.compile(loss='mse', optimizer='adam')

# fitting
history = model.fit(train_X, train_y, epochs=20, batch_size=20, validation_data=(test_X, test_y), verbose=1, shuffle=False)

# plot train and test performance/loss
plt.plot(history.history['loss'],'r')
plt.plot(history.history['val_loss'],'b')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['training', 'testing'])
plt.show();

In [ ]:
# prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], 9))
# invert scaling for forecast
inv_yhat = np.concatenate((test_X ,yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = pd.DataFrame(inv_yhat).iloc[:,9:]
# invert scaling for actual
###test_y = test_y.reshape((len(test_y), 7))
inv_y = np.concatenate((test_X, test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = pd.DataFrame(inv_y).iloc[:,9:]
# calculate RMSE, combine
temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k])))
rmse = np.sum(temp)/7    
print('Overall Test RMSE: %.3f' % rmse)

In [ ]:
inter=[x for x in range(inv_y.shape[0])]
plt.figure(figsize = (15,10))
plt.plot(inter, inv_y.iloc[:inv_y.shape[0],0], marker='.', label="actual")
plt.plot(inter, inv_yhat.iloc[:inv_y.shape[0],0], 'r', label="prediction")
plt.title('Predictions and actual values for t+1 step - all test data')
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

In [ ]:
inter=[x for x in range(30)]
plt.figure(figsize = (15,10))
plt.plot(inter, inv_y.iloc[:30,-1], marker='.', label="actual")
plt.plot(inter, inv_yhat.iloc[:30,-1], 'r', label="prediction")
plt.title('Predictions and actual values for t+1 step - 1 month')
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

In [ ]:
# Model 3: 7 lags, all features plus weather and weekend,
# input format [samples, timesteps, features] = [samples, 7, 9]
from sklearn.preprocessing import MinMaxScaler
data = df_sup_all.iloc[:,13:20]
temp_df_1 = pd.concat([df_sup_all.iloc[:,12:13],df_sup_all.iloc[:,32:33],df_sup_all.iloc[:,52:53],df_sup_all.iloc[:,72:73],df_sup_all.iloc[:,92:93],df_sup_all.iloc[:,112:113],df_sup_all.iloc[:,132:133],df_sup_all.iloc[:,152:153],df_sup_all.iloc[:,172:173]],axis = 1)
temp_df_2 = pd.concat([df_sup_all.iloc[:,11:12], df_sup_all.iloc[:,31:32],df_sup_all.iloc[:,51:52],df_sup_all.iloc[:,71:72],df_sup_all.iloc[:,91:92],df_sup_all.iloc[:,111:112],df_sup_all.iloc[:,131:132],df_sup_all.iloc[:,151:152],df_sup_all.iloc[:,171:172]],axis = 1)
temp_df_3 = pd.concat([df_sup_all.iloc[:,10:11], df_sup_all.iloc[:,30:31],df_sup_all.iloc[:,50:51],df_sup_all.iloc[:,70:71],df_sup_all.iloc[:,90:91],df_sup_all.iloc[:,110:111],df_sup_all.iloc[:,130:131],df_sup_all.iloc[:,150:151],df_sup_all.iloc[:,170:171]],axis = 1)
temp_df_4 = pd.concat([df_sup_all.iloc[:,9:10], df_sup_all.iloc[:,29:30],df_sup_all.iloc[:,49:50],df_sup_all.iloc[:,69:70],df_sup_all.iloc[:,89:90],df_sup_all.iloc[:,109:110],df_sup_all.iloc[:,129:130],df_sup_all.iloc[:,149:150],df_sup_all.iloc[:,169:170]],axis = 1)
temp_df_5 = pd.concat([df_sup_all.iloc[:,8:9], df_sup_all.iloc[:,28:29],df_sup_all.iloc[:,48:49],df_sup_all.iloc[:,68:69],df_sup_all.iloc[:,88:89],df_sup_all.iloc[:,108:109],df_sup_all.iloc[:,128:129],df_sup_all.iloc[:,148:149],df_sup_all.iloc[:,168:169]],axis = 1)
temp_df_6 = pd.concat([df_sup_all.iloc[:,7:8], df_sup_all.iloc[:,27:28],df_sup_all.iloc[:,47:48],df_sup_all.iloc[:,67:68],df_sup_all.iloc[:,87:88],df_sup_all.iloc[:,107:108],df_sup_all.iloc[:,127:128],df_sup_all.iloc[:,147:148],df_sup_all.iloc[:,167:168]],axis = 1)
temp_df_7 = pd.concat([df_sup_all.iloc[:,6:7], df_sup_all.iloc[:,26:27],df_sup_all.iloc[:,46:47],df_sup_all.iloc[:,66:67],df_sup_all.iloc[:,86:87],df_sup_all.iloc[:,106:107],df_sup_all.iloc[:,126:127],df_sup_all.iloc[:,146:147],df_sup_all.iloc[:,166:167]],axis = 1)
data = pd.concat([temp_df_7,temp_df_6,temp_df_5,temp_df_4,temp_df_3,temp_df_2,temp_df_1,data],axis = 1)
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)
data = pd.DataFrame(data)
n_train = int(0.5 * len(d))
train_X = data.iloc[:n_train,:63]
train_X = train_X.values.reshape(train_X.shape[0],7,9)
train_y = data.iloc[:n_train,63:]
test_X = data.iloc[n_train:,:63]
test_X = test_X.values.reshape(test_X.shape[0],7,9)
test_y = data.iloc[n_train:,63:]
train_X.shape, train_y.shape, test_X.shape, test_y.shape

In [ ]:
# modelling choices

model = Sequential()
model.add(LSTM(200,activation='relu',return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(100,activation = 'relu',return_sequences=False))
model.add(Dense(7))
model.compile(loss='mse', optimizer='adam')

# fitting
history = model.fit(train_X, train_y, epochs=15, batch_size=50, validation_data=(test_X, test_y), verbose=1, shuffle=False)

# plot train and test performance/loss
plt.plot(history.history['loss'],'r')
plt.plot(history.history['val_loss'],'b')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['training', 'testing'])
plt.show();

In [ ]:
# prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], 63))
# invert scaling for forecast
inv_yhat = np.concatenate((test_X ,yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = pd.DataFrame(inv_yhat).iloc[:,63:]
# invert scaling for actual
###test_y = test_y.reshape((len(test_y), 7))
inv_y = np.concatenate((test_X, test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = pd.DataFrame(inv_y).iloc[:,63:]
# calculate RMSE, combine
temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k])))
rmse = np.sum(temp)/7    
print('Overall Test RMSE: %.3f' % rmse)

In [ ]:
inter=[x for x in range(inv_y.shape[0])]
plt.figure(figsize = (15,10))
plt.plot(inter, inv_y.iloc[:inv_y.shape[0],0], marker='.', label="actual")
plt.plot(inter, inv_yhat.iloc[:inv_y.shape[0],0], 'r', label="prediction")
plt.title('Predictions and actual values for t+1 step - all test data')
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

In [ ]:
inter=[x for x in range(30)]
plt.figure(figsize = (15,10))
plt.plot(inter, inv_y.iloc[:30,-1], marker='.', label="actual")
plt.plot(inter, inv_yhat.iloc[:30,-1], 'r', label="prediction")
plt.title('Predictions and actual values for t+1 step - 1 month')
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

<br>
<div class="alert alert-success">
<b>8. OPTIONAL ASSESSMENT (~200 words)</b> 

<p> Put your narrative or explanatory text in this cell (or additional cells as necessary) <p>

</div>

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>
<br>
<br>
Models 2 and 3: based on the data engineering in the supervised framing section. Model 2 is an LSTM that feeds in only 1 time lag but all available 9 features. The input structure is [#samples, timesteps, features] = [#samples, 1, 9]. It uses two LSTM layers and a 0.5 dropout to predict 7 outcomes (t+1…t+7). 

Model 3: LSTM that feeds in 7 time lag but all available variables. The input structure is [#samples, 7, 9]. The architecture is similar. Model 2 reaches an average RMSE of 553 which increases fast in predicted time steps. Model 3, although build on more lags, performs similar but the RMSE does not increase (from t+1 to t+7), therefore it is a more stable model.

All models show a higher training loss compared to the testing loss (see graphs above) which logically does not make sense to us although it might have to do with the drop-outs or how Keras handles training and testing differently. Also, we are convinced that the neural nets would perform better if they would have been trained on an appropriate amount of data (we only have 728 training samples available in this case study). 

<br>
<br>
Word count: 195

</div>

<br>
<div class="alert alert-success">
<h1 align="center"> 5. Algorithm further assesment </h1> <br>
<a id="assesment"></a>

## Evaluation

You have trained more that two different regressors using different methods and tested it on different dataset. 

Comment on the comparative performance of the methods your have investigated. 

In [77]:
# This section contains:
# 1) GBM - Experiment: Change the train test-test split from 50/50 to 60/40 and show that lack of data might be an issue
# 2) GBM - Plot true vs. predicted values and visualise the performance of the best multi-step model
# 3) Take the difference between the predictions and the true values of the t+1 step of the 7 day multi-step GBM system
# 4) The mark up cell gives an overview about comparability of results

In [ ]:
# 1) GBM - Experiment: Change the train test-test split from 0.5/0.5 to 0.6/0.4 and show that lack of data is an issue
# Create a direct modelling approach

data = df_sup_all.iloc[:,13:20]
temp_df = pd.concat([df_sup_all.iloc[:,0:13], df_sup_all.iloc[:,20:33],df_sup_all.iloc[:,40:53],df_sup_all.iloc[:,60:73],df_sup_all.iloc[:,80:93],df_sup_all.iloc[:,100:113],df_sup_all.iloc[:,120:133],df_sup_all.iloc[:,140:153],df_sup_all.iloc[:,160:173]],axis = 1)
df_direct = pd.concat([temp_df,data],axis = 1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df_direct = scaler.fit_transform(df_direct)
df_direct = pd.DataFrame(df_direct)
n_train = int(0.6 * len(d))
train_X = df_direct.iloc[:n_train,:117]
train_y = df_direct.iloc[:n_train,117:]
test_X = df_direct.iloc[n_train:,:117]
test_y = df_direct.iloc[n_train:,117:]
#train_X.shape, train_y.shape, test_X.shape, test_y.shape

# GRADIENT BOOSTING - MULTI-STEP 7 DAYS

from sklearn.ensemble import GradientBoostingRegressor

yhat = pd.DataFrame()
for k in range(7):
    model = GradientBoostingRegressor().fit(train_X, train_y.iloc[:,k])
    yhat['rf_'+str(k)] = model.predict(test_X)
    
inv_yhat = np.concatenate((test_X ,yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = pd.DataFrame(inv_yhat).iloc[:,117:]

inv_y = np.concatenate((test_X, test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = pd.DataFrame(inv_y).iloc[:,117:]

gbm_inv_y_save = inv_y.iloc[:,-1]
gbm_inv_yhat_save = inv_yhat.iloc[:,-1]

temp = list()
for k in range(7):
    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k]))))
    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k])))
rmse = np.sum(temp)/7    
print('----------------------------')
print('Overall Test RMSE with the 60/40 split: %.3f' % rmse)
print('Compare: Overall Test RMSE with the 50/50 split: 523ish')

In [79]:
#----------------------------
#Overall Test RMSE with the 60/40 split: 506.052
#Compare: Overall Test RMSE with the 50/50 split: 523ish

In [ ]:
# 2) True values vs. predictions - still far from a 45 degree line
plt.figure(figsize = (9,9))
plt.scatter(gbm_inv_y_save,gbm_inv_yhat_save)
plt.title('GBM (t+1 step): True values vs. predictions', fontsize = 15)
plt.ylabel('True values for t+1 (global active power)', size=12)
plt.xlabel('Predicted values for t+1 (global active power)', size=12)
plt.show();

In [ ]:
# 3) Take the difference between the predictions and the true values of the t+1 step of the 7 day multi-step GBM system
# Note: The following statistics are based on the model above (60/40 split).

In [ ]:
diff_gbm = gbm_inv_yhat_save-gbm_inv_y_save

In [ ]:
# mean != median -> slightly right-skewed distribution of residuals
diff_gbm.mean(), diff_gbm.median()
# For comparision:
# The experiment in this section (60/40 split) results in: (-84.15365986985809, -82.67020167824012)
# The regular model in the notebook (50/50 split) results in: (-151.80721716904435, -128.53251884434917)
# The mean residual is twice as high!!

In [ ]:
# Plot the residuals - the shape looks quite good although the upper tail is irregular 
# The residuals are distributed quite symmetric but the variance of the distribution is quite big, especially see the long tails!
# This makes the model costly for us.
diff_gbm.plot(kind='kde', color='grey');

In [ ]:
# See the quantiles
diff_gbm.describe()

<br>
<div class="alert alert-warning">
<b>9. ASSESSMENT (~200 words)</b> 

<p> Put your narrative or explanatory text in this cell (or additional cells as necessary) <p>

</div>

<div class="alert alert-block alert-info">
    
<b>SECTION COMPREHENSION</b>
<br>
<br>
All models are based on the same train-test split. Some models (ARIMA and one LSTM) are doing one-step forecasting, other models (supervised base nd ensembles, 2 LSTMs) are performing multi-step forecast. The choice of the persistence model could have been different as well (e.g. use t-1 instead of t-7 for prediciton of t). The most severe issue we see is the lack of data: We work on a 728 sample train set which is not much to train an algorithm, especially not the LSTMs. The experiment above shows that a 60/40 (instead of 50/50) split decreases RMSE for GMB by 20 already. Second, the data structure changes in the last year and models might be not able to generalise sufficiently. Third, some models are built on more lags and more features than others. Models after the supervised section are comparable. The plot above shows that the predictions vs. true values for GMB do not form a 45 degree line. The result shows that there is lots of potential to improve the predicitve ability of our system. 
<br>
<br>
Word count: 176

</div>

<br>
<div class="alert alert-success">
<h1 align="center"> 6. Finalize Model and Summary </h1> <br>
<a id="final"></a>

In [ ]:
#neural network model:
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# make a prediction
yhat = model.predict(test_X)
    
#it can be accest throught the function:
#network_model(data, train_test_split, column of the chose of estimation)
#it was run in the 7th section of thisnotbook, all results avaliable there
    
    

In [ ]:
%%time
# GRID SEARCH VROSS VALIDATION OF GRADIENT BOOSTING

# !!!
# NOTE: DO NOT RUN. We used GridSreachCV to find optimal parameters. It runs for 30 min but the results are not better than the default settings.
# So for a final model, see the GBM model in the ensemble section of this notebook. RMSE: 522 (7 days multi-step, direct modeling approach)
# !!!

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# By using the supervised data frame, the time-variant/dependent information for each point in time is captured in each row, therefore the rows itself
# become time-independent. We can therefore use standard cross-validation with a grid search to find the optimal parameters for all 7 models.

#  set paramter ranges
parameters = {
    "loss":["ls"],
    "learning_rate": [0.05, 0.1, 0.2],
    "max_depth":[2,3,5],
    "min_samples_split":[2,4,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse"],
    "subsample":[0.7, 0.85, 0.9],
    "n_estimators":[100, 200, 500]
    }

#note that the following code runs 
#optimised_models = list()
#yhat = pd.DataFrame()
#for k in range(7):
#    model = GridSearchCV(GradientBoostingRegressor(), parameters,scoring='neg_mean_squared_error',cv=5)
#    model.fit(train_X, train_y.iloc[:,k])
#    yhat['gb_'+str(k)] = model.predict(test_X)
#    print('gb_'+str(k)+' GridSearchCV successful')
#    optimised_models.append(model)
    
#print('----------------------------')
#inv_yhat = np.concatenate((test_X ,yhat), axis=1)
#inv_yhat = scaler.inverse_transform(inv_yhat)
#inv_yhat = pd.DataFrame(inv_yhat).iloc[:,117:]

#inv_y = np.concatenate((test_X, test_y), axis=1#)
#inv_y = scaler.inverse_transform(inv_y)
#inv_y = pd.DataFrame(inv_y).iloc[:,117:]

#temp = list()
#for k in range(7):
#    print('RMSE t+'+str(k+1)+': '+str(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k]))))
#    temp.append(np.sqrt(mean_squared_error(inv_y.iloc[:,k], inv_yhat.iloc[:,k])))
#rmse = np.sum(temp)/7    
#print('----------------------------')
#print('Overall Test RMSE: %.3f' % rmse)

#inter=[x for x in range(inv_y.shape[0])]
#plt.figure(figsize = (10,7))
#plt.plot(inter, inv_y.iloc[:inv_y.shape[0],0], marker='.', label="actual")
#plt.plot(inter, inv_yhat.iloc[:inv_y.shape[0],0], 'r', label="prediction")
#plt.title('Predictions and actual values for t+1 step - all test data')
#plt.ylabel('Global_active_power', size=15)
#plt.xlabel('Time step', size=15)
#plt.legend(fontsize=15)
#plt.show();

In [ ]:
# all seven models available here, just print the first
#optimised_models[0]

<div class="alert alert-warning">
<b>Summary: </b> <br>
    
As our analysis showed following models were most accurate in their classes:<br>
- The ARIMA yields an RMSE of <b>671</b>.<br>
- The best model overall: ensamble gradient boosted with RMSE of <b>525</b>.<br>
- The best supervised model: polynomial Ridge regression wiht RMSE of <b>899</b>.<br>
- The best NN model: Model 3 with 7 lags, all features plus weather and weekend with RMSE of <b>538</b>.<br>
- ARIMA performance with RMSE of <b>696</b>.<br>
- The evualuation shows that more data would have been needed for training of the models
- Using more than 7 days as time lags might increase the predictive ability of the system
<br>
<br>
Word count: 90

</div>

<div class="alert alert-warning">
<b>Sources: </b> <br>
Code for Neural Networks:  <br>
https://www.kaggle.com/amirrezaeian/time-series-data-analysis-using-lstm-tutorial <br>
https://machinelearningmastery.com/how-to-develop-lstm-models-for-multi-step-time-series-forecasting-of-household-power-consumption/<br>

</div>

<div class="alert alert-block alert-info">
    
<b>WORD COUNT</b>
<br>
<br>
Notebook 1: 634 <br>
Notebook 2: 1200<br>
<br>
Total word count: < 2000 

</div>